In [1]:
import gsapi
import numpy as np
import pandas as pd
import datetime as dt

### Reading data from a Google Sheets document using the Google Sheets API

In [2]:
# load_data - the function of reading data from a Google Sheets document
def load_data(spreadsheet_id, sheet_and_range):
    data = gsapi.read_data(spreadsheet_id,sheet_and_range)
    return pd.DataFrame(data[1:], columns=data[0])

In [3]:
spreadsheet_id = '1Ycg7zTxds9DZnDvTrFcyNNKuTUxg6Yy6WF0a8Wc02WQ' # document id
list_sheets = ['transactions!A:D', 'clients!A:C','managers!A:C', 'leads!A:F'] # range of loaded data from each page

In [4]:
# reading the necessary data from the pages of the document
transactions = load_data(spreadsheet_id,list_sheets[0])
clients = load_data(spreadsheet_id,list_sheets[1])
managers = load_data(spreadsheet_id,list_sheets[2])
leads = load_data(spreadsheet_id,list_sheets[3])

### Cleaning and processing "transactions" data

In [5]:
# clean - data cleaning function
def clean(data):
    for i in data.columns:
        data[i] = data[i].replace({'0001-01-01 00:00:00':np.nan})
        data[i] = data[i].replace({'00000000-0000-0000-0000-000000000000':np.nan})
        data[i] = data[i].replace({'':np.nan,'ig':'instagram','vkontakte':'vk',
                                  'ycard#!/tproduct/225696739-1498486363994':'ycard',
                                  'insta':'instagram'})

In [6]:
clean(transactions)
transactions = transactions.rename(columns={"created_at":"tran_created_at", "l_client_id":"client_id"})
transactions['m_real_amount'] = transactions['m_real_amount'].astype(int)
transactions["tran_created_at"] = pd.to_datetime(transactions["tran_created_at"])

In [7]:
transactions = transactions.drop_duplicates()

In [8]:
print(transactions.shape)

(28987, 4)


In [9]:
transactions.head()

,transaction_id,tran_created_at,m_real_amount,client_id
0,2c9f1527-8e7f-4fb1-8000-c747a2ab46c9,2020-09-30 07:15:14,31,8a805e60-6fd6-11e7-80fc-c412f533dba1
1,2c9f1527-8e7f-4fb1-8000-c747a2ab46c9,2020-09-30 07:15:14,87,8a805e60-6fd6-11e7-80fc-c412f533dba1
2,ab8cbcf7-3271-49a0-8001-6cf9816f63b8,2020-09-12 13:47:04,49231,dc0d0e52-629a-11ea-abf3-c412f533dba1
3,ab8cbcf7-3271-49a0-8001-6cf9816f63b8,2020-09-12 13:47:04,17305,dc0d0e52-629a-11ea-abf3-c412f533dba1
4,52f8ebcc-d82d-4be1-8004-72cfbe0dff24,2020-09-25 06:00:07,9022,e663b6a0-4a1a-11e8-a2c3-c412f533dba1


### Cleaning and processing "clients" data

In [10]:
clean(clients)
clients = clients.rename(columns={"created_at":"client_created_at", "l_manager_id":"manager_id"})
clients['client_created_at'] = pd.to_datetime(clients['client_created_at'])

In [11]:
clients = clients.drop_duplicates()

In [12]:
print(clients.shape)

(75766, 3)


In [13]:
clients.head()

,client_id,client_created_at,manager_id
0,0001da7d-fcda-11ea-ac18-c412f533dba1,2020-09-22 17:46:23,9a87c608-52dc-11ea-abeb-c412f533dba1
1,000a8743-9ae1-11e7-8114-c412f533dba1,2017-09-16 00:00:00,ad52c7a8-a752-11e7-8115-c412f533dba1
2,000ae57e-2d48-11ea-abeb-c412f533dba1,2020-01-02 14:08:56,43756fa4-57a0-11e9-ab9a-c412f533dba1
3,00133cde-481c-11ea-abeb-c412f533dba1,2020-02-05 17:25:13,9a87c608-52dc-11ea-abeb-c412f533dba1
4,00148d3f-07e6-11e8-812a-c412f533dba1,2018-02-02 00:00:00,abed5496-ca88-11e9-abb8-c412f533dba1


### Cleaning and processing “leads” data

In [14]:
clean(leads)
leads = leads.rename(columns={"created_at":"lead_created_at", "l_manager_id":"manager_id", "l_client_id":"client_id"})
leads["lead_created_at"] = pd.to_datetime(leads["lead_created_at"])

In [15]:
leads = leads.drop_duplicates()

In [16]:
print(leads.shape)

(3337, 6)


In [17]:
# adding columns "d_manager" and "d_club"
leads = leads.merge(managers, how='left', on='manager_id')

In [18]:
print(leads.shape)
leads.head()

(3337, 8)


,lead_id,lead_created_at,d_utm_medium,d_utm_source,manager_id,client_id,d_manager,d_club
0,00678e71-0fb9-11eb-ac1a-c412f533dba1,2020-10-16 18:08:09,sms,viber,1e9c5562-0cfc-11ea-abe1-c412f533dba1,7264cf21-a31d-11e9-abb0-c412f533dba1,manager #8,club #2
1,0078b05d-0f1f-11eb-ac1a-c412f533dba1,2020-10-15 23:45:40,cpc,instagram,d12fe765-b444-11e9-abb3-c412f533dba1,606c5b29-0f88-11eb-ac1a-c412f533dba1,manager #1,club #1
2,007d1401-2af3-11eb-ac1f-c412f533dba1,2020-11-20 09:41:14,cpc,instagram,43756fa4-57a0-11e9-ab9a-c412f533dba1,1f389c6a-d5ab-11e9-abbe-c412f533dba1,manager #11,club #4
3,0084e614-2fcc-11eb-ac1f-c412f533dba1,2020-11-26 13:44:40,NaN,NaN,NaN,NaN,NaN,NaN
4,009b1616-145d-11eb-ac1d-c412f533dba1,2020-10-22 15:52:04,cpc,vk,5dd88f32-6501-11ea-abf3-c412f533dba1,ac2c0012-6fd5-11e7-80fc-c412f533dba1,manager #10,club #2


### Generating metrics for the final report

#### lead - number of orders

In [19]:
leads['lead'] = 1

In [20]:
print(leads.shape)
leads.head()

(3337, 9)


,lead_id,lead_created_at,d_utm_medium,d_utm_source,manager_id,client_id,d_manager,d_club,lead
0,00678e71-0fb9-11eb-ac1a-c412f533dba1,2020-10-16 18:08:09,sms,viber,1e9c5562-0cfc-11ea-abe1-c412f533dba1,7264cf21-a31d-11e9-abb0-c412f533dba1,manager #8,club #2,1
1,0078b05d-0f1f-11eb-ac1a-c412f533dba1,2020-10-15 23:45:40,cpc,instagram,d12fe765-b444-11e9-abb3-c412f533dba1,606c5b29-0f88-11eb-ac1a-c412f533dba1,manager #1,club #1,1
2,007d1401-2af3-11eb-ac1f-c412f533dba1,2020-11-20 09:41:14,cpc,instagram,43756fa4-57a0-11e9-ab9a-c412f533dba1,1f389c6a-d5ab-11e9-abbe-c412f533dba1,manager #11,club #4,1
3,0084e614-2fcc-11eb-ac1f-c412f533dba1,2020-11-26 13:44:40,NaN,NaN,NaN,NaN,NaN,NaN,1
4,009b1616-145d-11eb-ac1d-c412f533dba1,2020-10-22 15:52:04,cpc,vk,5dd88f32-6501-11ea-abf3-c412f533dba1,ac2c0012-6fd5-11e7-80fc-c412f533dba1,manager #10,club #2,1


#### trash_lead - number of junk orders

In [21]:
leads['trash_lead'] = leads.client_id.isna().astype('int')

In [22]:
print(leads.shape)
leads.head(3)

(3337, 10)


,lead_id,lead_created_at,d_utm_medium,d_utm_source,manager_id,client_id,d_manager,d_club,lead,trash_lead
0,00678e71-0fb9-11eb-ac1a-c412f533dba1,2020-10-16 18:08:09,sms,viber,1e9c5562-0cfc-11ea-abe1-c412f533dba1,7264cf21-a31d-11e9-abb0-c412f533dba1,manager #8,club #2,1,0
1,0078b05d-0f1f-11eb-ac1a-c412f533dba1,2020-10-15 23:45:40,cpc,instagram,d12fe765-b444-11e9-abb3-c412f533dba1,606c5b29-0f88-11eb-ac1a-c412f533dba1,manager #1,club #1,1,0
2,007d1401-2af3-11eb-ac1f-c412f533dba1,2020-11-20 09:41:14,cpc,instagram,43756fa4-57a0-11e9-ab9a-c412f533dba1,1f389c6a-d5ab-11e9-abbe-c412f533dba1,manager #11,club #4,1,0


#### new_lead - number of new orders (there were no orders and purchases from this client before)

In [23]:
min_date = leads.lead_created_at.min()
print(min_date)

2020-10-01 00:02:55


In [24]:
# generating new customers based on purchase data
t = transactions[transactions.tran_created_at < min_date]
new_client_t = pd.merge(leads,t[['client_id','tran_created_at']],how='inner')
new_client_t['new_client_t'] = 1
new_client_t = new_client_t[['client_id','new_client_t']]

In [25]:
# generating new clients based on data on previously registered clients
c = clients[clients.client_created_at < min_date]
new_client_c = pd.merge(leads,c[['client_id','client_created_at']],how='inner')
new_client_c['new_client_c'] = 1
new_client_c = new_client_c[['client_id','new_client_c']]

In [26]:
leads = pd.merge(leads,new_client_t, how='left').drop_duplicates()
leads = pd.merge(leads,new_client_c, how='left').drop_duplicates()

In [27]:
# creating the 'new_lead' column
new_client = ~(leads[['new_client_t','new_client_c']].sum(axis=1) > 0)
leads['new_lead'] = new_client.astype(int)
leads.drop(['new_client_t', 'new_client_c'], axis=1, inplace=True)

In [28]:
print(leads.shape)
leads.head()

(3337, 11)


,lead_id,lead_created_at,d_utm_medium,d_utm_source,manager_id,client_id,d_manager,d_club,lead,trash_lead,new_lead
0,00678e71-0fb9-11eb-ac1a-c412f533dba1,2020-10-16 18:08:09,sms,viber,1e9c5562-0cfc-11ea-abe1-c412f533dba1,7264cf21-a31d-11e9-abb0-c412f533dba1,manager #8,club #2,1,0,0
1,0078b05d-0f1f-11eb-ac1a-c412f533dba1,2020-10-15 23:45:40,cpc,instagram,d12fe765-b444-11e9-abb3-c412f533dba1,606c5b29-0f88-11eb-ac1a-c412f533dba1,manager #1,club #1,1,0,1
2,007d1401-2af3-11eb-ac1f-c412f533dba1,2020-11-20 09:41:14,cpc,instagram,43756fa4-57a0-11e9-ab9a-c412f533dba1,1f389c6a-d5ab-11e9-abbe-c412f533dba1,manager #11,club #4,1,0,1
3,0084e614-2fcc-11eb-ac1f-c412f533dba1,2020-11-26 13:44:40,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1
4,009b1616-145d-11eb-ac1d-c412f533dba1,2020-10-22 15:52:04,cpc,vk,5dd88f32-6501-11ea-abf3-c412f533dba1,ac2c0012-6fd5-11e7-80fc-c412f533dba1,manager #10,club #2,1,0,0


#### buyer - number of buyers (those who bought within a week after the order)

In [29]:
# selecting new transactions where the purchase was made
pur = transactions[transactions.m_real_amount > 0]
pur = pur[pur.tran_created_at >= min_date]
print(pur.shape)
pur.head()

(5738, 4)


,transaction_id,tran_created_at,m_real_amount,client_id
7,704e066b-7b49-4439-8005-d412e4d1c18b,2020-10-13 05:59:02,4405,a9bcf7ac-d3dd-11e9-abbe-c412f533dba1
9,61e4c410-6765-4d5d-8006-873fe1ec9c2d,2020-11-25 12:57:13,41369,64529cdc-d5b2-11e9-abbe-c412f533dba1
10,f904046c-73d7-43fa-8006-beadc89ecf2f,2020-11-23 13:22:39,16005,cea2f440-2d85-11eb-ac1f-c412f533dba1
17,3524d29d-6778-43fd-8007-eae40afdb39d,2020-11-27 15:50:44,25175,dc7d0bc9-6fd5-11e7-80fc-c412f533dba1
29,d2e4989c-dd2a-4230-800d-8066d9ffb126,2020-11-14 13:26:08,48412,a9259f72-d5a5-11e9-abbe-c412f533dba1


In [30]:
# selecting transactions that were made after the order was submitted by the client
pur = pd.merge(pur,leads[['client_id','lead_created_at','lead_id']],how='inner')
pur = pur[pur.tran_created_at > pur.lead_created_at]
print(pur.shape)
pur.head()

(974, 6)


,transaction_id,tran_created_at,m_real_amount,client_id,lead_created_at,lead_id
1,d2e4989c-dd2a-4230-800d-8066d9ffb126,2020-11-14 13:26:08,48412,a9259f72-d5a5-11e9-abbe-c412f533dba1,2020-11-13 12:41:23,020a526c-258c-11eb-ac1d-c412f533dba1
2,ef894648-ad3e-4701-801f-e8403dd6177b,2020-10-20 16:29:17,56094,dc018616-1085-11eb-ac1a-c412f533dba1,2020-10-17 17:54:07,38e0730d-1080-11eb-ac1a-c412f533dba1
3,ef894648-ad3e-4701-801f-e8403dd6177b,2020-10-20 16:29:17,56094,dc018616-1085-11eb-ac1a-c412f533dba1,2020-10-17 18:02:58,7584a93f-1081-11eb-ac1a-c412f533dba1
4,ef894648-ad3e-4701-801f-e8403dd6177b,2020-10-20 16:29:17,56094,dc018616-1085-11eb-ac1a-c412f533dba1,2020-10-17 18:05:38,d5102838-1081-11eb-ac1a-c412f533dba1
5,a7ea159a-7fd0-465e-804f-0fb6208a54af,2020-10-20 16:35:57,11252,b4bb26da-06d2-11eb-ac18-c412f533dba1,2020-10-03 07:24:40,f8d24a27-0527-11eb-ac18-c412f533dba1


In [31]:
# selecting transactions that were made within a week after the order was submitted by the client
week = dt.timedelta(days=7)
pur = pur[(pur.tran_created_at - pur.lead_created_at) < week]
pur = pur.drop_duplicates(subset=['transaction_id','tran_created_at','m_real_amount'], keep='last')
print(pur.shape)
pur.head()

(464, 6)


,transaction_id,tran_created_at,m_real_amount,client_id,lead_created_at,lead_id
1,d2e4989c-dd2a-4230-800d-8066d9ffb126,2020-11-14 13:26:08,48412,a9259f72-d5a5-11e9-abbe-c412f533dba1,2020-11-13 12:41:23,020a526c-258c-11eb-ac1d-c412f533dba1
4,ef894648-ad3e-4701-801f-e8403dd6177b,2020-10-20 16:29:17,56094,dc018616-1085-11eb-ac1a-c412f533dba1,2020-10-17 18:05:38,d5102838-1081-11eb-ac1a-c412f533dba1
6,a823495a-4501-490b-8051-b19ca01989ac,2020-10-28 17:20:03,74362,17bc9490-d5a7-11e9-abbe-c412f533dba1,2020-10-25 10:49:30,3b42b680-168e-11eb-ac1d-c412f533dba1
8,25810c12-86cb-4c00-8076-79b5cb1c0c37,2020-11-22 11:46:39,76084,da5efc83-2740-11eb-ac1d-c412f533dba1,2020-11-15 16:48:26,da5efc82-2740-11eb-ac1d-c412f533dba1
11,233681f8-2b6c-4106-8088-04a6997a5ab4,2020-10-16 14:44:44,11582,9cccebaa-0e25-11eb-ac1a-c412f533dba1,2020-10-14 17:49:51,216bc10e-0e24-11eb-ac1a-c412f533dba1


In [32]:
# сreating the 'buyer' column
buyers = pur.groupby('client_id').m_real_amount.sum()
buyers = buyers.reset_index()
leads = pd.merge(leads, buyers, how='left')
leads['buyer'] = leads.m_real_amount.notna().astype(int)

In [33]:
print(leads.shape)
leads.head()

(3337, 13)


,lead_id,lead_created_at,d_utm_medium,d_utm_source,manager_id,client_id,d_manager,d_club,lead,trash_lead,new_lead,m_real_amount,buyer
0,00678e71-0fb9-11eb-ac1a-c412f533dba1,2020-10-16 18:08:09,sms,viber,1e9c5562-0cfc-11ea-abe1-c412f533dba1,7264cf21-a31d-11e9-abb0-c412f533dba1,manager #8,club #2,1,0,0,44041.0,1
1,0078b05d-0f1f-11eb-ac1a-c412f533dba1,2020-10-15 23:45:40,cpc,instagram,d12fe765-b444-11e9-abb3-c412f533dba1,606c5b29-0f88-11eb-ac1a-c412f533dba1,manager #1,club #1,1,0,1,NaN,0
2,007d1401-2af3-11eb-ac1f-c412f533dba1,2020-11-20 09:41:14,cpc,instagram,43756fa4-57a0-11e9-ab9a-c412f533dba1,1f389c6a-d5ab-11e9-abbe-c412f533dba1,manager #11,club #4,1,0,1,NaN,0
3,0084e614-2fcc-11eb-ac1f-c412f533dba1,2020-11-26 13:44:40,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1,NaN,0
4,009b1616-145d-11eb-ac1d-c412f533dba1,2020-10-22 15:52:04,cpc,vk,5dd88f32-6501-11ea-abf3-c412f533dba1,ac2c0012-6fd5-11e7-80fc-c412f533dba1,manager #10,club #2,1,0,0,NaN,0


#### new_buyer - number of new buyers (those who bought within a week after the order, and had not bought earlier)

In [34]:
new_bayer = leads[['new_lead','buyer']].sum(axis=1)>1
leads['new_buyer'] = new_bayer.astype(int)

In [35]:
print(leads.shape)
leads.head()

(3337, 14)


,lead_id,lead_created_at,d_utm_medium,d_utm_source,manager_id,client_id,d_manager,d_club,lead,trash_lead,new_lead,m_real_amount,buyer,new_buyer
0,00678e71-0fb9-11eb-ac1a-c412f533dba1,2020-10-16 18:08:09,sms,viber,1e9c5562-0cfc-11ea-abe1-c412f533dba1,7264cf21-a31d-11e9-abb0-c412f533dba1,manager #8,club #2,1,0,0,44041.0,1,0
1,0078b05d-0f1f-11eb-ac1a-c412f533dba1,2020-10-15 23:45:40,cpc,instagram,d12fe765-b444-11e9-abb3-c412f533dba1,606c5b29-0f88-11eb-ac1a-c412f533dba1,manager #1,club #1,1,0,1,NaN,0,0
2,007d1401-2af3-11eb-ac1f-c412f533dba1,2020-11-20 09:41:14,cpc,instagram,43756fa4-57a0-11e9-ab9a-c412f533dba1,1f389c6a-d5ab-11e9-abbe-c412f533dba1,manager #11,club #4,1,0,1,NaN,0,0
3,0084e614-2fcc-11eb-ac1f-c412f533dba1,2020-11-26 13:44:40,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1,NaN,0,0
4,009b1616-145d-11eb-ac1d-c412f533dba1,2020-10-22 15:52:04,cpc,vk,5dd88f32-6501-11ea-abf3-c412f533dba1,ac2c0012-6fd5-11e7-80fc-c412f533dba1,manager #10,club #2,1,0,0,NaN,0,0


#### new_m_real_amount - income from new buyers’ purchases

In [36]:
leads['new_m_real_amount'] = leads.new_buyer * leads.m_real_amount

In [37]:
print(leads.shape)
leads.head()

(3337, 15)


,lead_id,lead_created_at,d_utm_medium,d_utm_source,manager_id,client_id,d_manager,d_club,lead,trash_lead,new_lead,m_real_amount,buyer,new_buyer,new_m_real_amount
0,00678e71-0fb9-11eb-ac1a-c412f533dba1,2020-10-16 18:08:09,sms,viber,1e9c5562-0cfc-11ea-abe1-c412f533dba1,7264cf21-a31d-11e9-abb0-c412f533dba1,manager #8,club #2,1,0,0,44041.0,1,0,0.0
1,0078b05d-0f1f-11eb-ac1a-c412f533dba1,2020-10-15 23:45:40,cpc,instagram,d12fe765-b444-11e9-abb3-c412f533dba1,606c5b29-0f88-11eb-ac1a-c412f533dba1,manager #1,club #1,1,0,1,NaN,0,0,NaN
2,007d1401-2af3-11eb-ac1f-c412f533dba1,2020-11-20 09:41:14,cpc,instagram,43756fa4-57a0-11e9-ab9a-c412f533dba1,1f389c6a-d5ab-11e9-abbe-c412f533dba1,manager #11,club #4,1,0,1,NaN,0,0,NaN
3,0084e614-2fcc-11eb-ac1f-c412f533dba1,2020-11-26 13:44:40,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1,NaN,0,0,NaN
4,009b1616-145d-11eb-ac1d-c412f533dba1,2020-10-22 15:52:04,cpc,vk,5dd88f32-6501-11ea-abf3-c412f533dba1,ac2c0012-6fd5-11e7-80fc-c412f533dba1,manager #10,club #2,1,0,0,NaN,0,0,NaN


In [38]:
# filling missing values
leads = leads.fillna(0)
print(leads.shape)
leads.head()

(3337, 15)


,lead_id,lead_created_at,d_utm_medium,d_utm_source,manager_id,client_id,d_manager,d_club,lead,trash_lead,new_lead,m_real_amount,buyer,new_buyer,new_m_real_amount
0,00678e71-0fb9-11eb-ac1a-c412f533dba1,2020-10-16 18:08:09,sms,viber,1e9c5562-0cfc-11ea-abe1-c412f533dba1,7264cf21-a31d-11e9-abb0-c412f533dba1,manager #8,club #2,1,0,0,44041.0,1,0,0.0
1,0078b05d-0f1f-11eb-ac1a-c412f533dba1,2020-10-15 23:45:40,cpc,instagram,d12fe765-b444-11e9-abb3-c412f533dba1,606c5b29-0f88-11eb-ac1a-c412f533dba1,manager #1,club #1,1,0,1,0.0,0,0,0.0
2,007d1401-2af3-11eb-ac1f-c412f533dba1,2020-11-20 09:41:14,cpc,instagram,43756fa4-57a0-11e9-ab9a-c412f533dba1,1f389c6a-d5ab-11e9-abbe-c412f533dba1,manager #11,club #4,1,0,1,0.0,0,0,0.0
3,0084e614-2fcc-11eb-ac1f-c412f533dba1,2020-11-26 13:44:40,0,0,0,0,0,0,1,1,1,0.0,0,0,0.0
4,009b1616-145d-11eb-ac1d-c412f533dba1,2020-10-22 15:52:04,cpc,vk,5dd88f32-6501-11ea-abf3-c412f533dba1,ac2c0012-6fd5-11e7-80fc-c412f533dba1,manager #10,club #2,1,0,0,0.0,0,0,0.0


### Final table and exporting data to a Google Sheets document using the Google Sheets API

In [39]:
# converting dates to days
leads['lead_created_at'] = leads['lead_created_at'].dt.date
leads['lead_created_at'] = leads['lead_created_at'].astype(str)
leads.head()

,lead_id,lead_created_at,d_utm_medium,d_utm_source,manager_id,client_id,d_manager,d_club,lead,trash_lead,new_lead,m_real_amount,buyer,new_buyer,new_m_real_amount
0,00678e71-0fb9-11eb-ac1a-c412f533dba1,2020-10-16,sms,viber,1e9c5562-0cfc-11ea-abe1-c412f533dba1,7264cf21-a31d-11e9-abb0-c412f533dba1,manager #8,club #2,1,0,0,44041.0,1,0,0.0
1,0078b05d-0f1f-11eb-ac1a-c412f533dba1,2020-10-15,cpc,instagram,d12fe765-b444-11e9-abb3-c412f533dba1,606c5b29-0f88-11eb-ac1a-c412f533dba1,manager #1,club #1,1,0,1,0.0,0,0,0.0
2,007d1401-2af3-11eb-ac1f-c412f533dba1,2020-11-20,cpc,instagram,43756fa4-57a0-11e9-ab9a-c412f533dba1,1f389c6a-d5ab-11e9-abbe-c412f533dba1,manager #11,club #4,1,0,1,0.0,0,0,0.0
3,0084e614-2fcc-11eb-ac1f-c412f533dba1,2020-11-26,0,0,0,0,0,0,1,1,1,0.0,0,0,0.0
4,009b1616-145d-11eb-ac1d-c412f533dba1,2020-10-22,cpc,vk,5dd88f32-6501-11ea-abf3-c412f533dba1,ac2c0012-6fd5-11e7-80fc-c412f533dba1,manager #10,club #2,1,0,0,0.0,0,0,0.0


In [40]:
dimensions = ['lead_created_at','d_utm_source','d_club','d_manager']
metrics = ['lead','trash_lead','new_lead','buyer','new_buyer','new_m_real_amount']

In [41]:
df = leads[dimensions+metrics]
df = df.sort_values(by=dimensions)
print(df.shape)
df.head()

(3337, 10)


,lead_created_at,d_utm_source,d_club,d_manager,lead,trash_lead,new_lead,buyer,new_buyer,new_m_real_amount
2182,2020-10-01,0,0,0,1,1,1,0,0,0.0
290,2020-10-01,0,club #1,manager #1,1,1,1,0,0,0.0
153,2020-10-01,0,club #2,manager #8,1,0,0,0,0,0.0
84,2020-10-01,0,club #4,manager #2,1,0,0,0,0,0.0
36,2020-10-01,instagram,0,0,1,0,1,1,1,75560.0


In [42]:
column_name = [df.columns.tolist()]
push_data = df.values.tolist()

In [43]:
# document id
spreadsheet_id = '1xWas0rwKUqqll8M1GoMmwK8xByAKfoj4wmtscbQjI7g'

# range of exported data
sheet_range1 = 'dataframe!A1:J'
sheet_range2 = 'dataframe!A2:J'

In [44]:
# exporting data to a Google Sheets document
gsapi.push_data(spreadsheet_id, sheet_range1, column_name)

In [45]:
gsapi.push_data(spreadsheet_id, sheet_range2, push_data)

### Pivot table: 
https://docs.google.com/spreadsheets/d/1xWas0rwKUqqll8M1GoMmwK8xByAKfoj4wmtscbQjI7g/edit#gid=1859765491